In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
from collections import defaultdict
from collections import deque
import nltk
import re
import os
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **PRE-PROCESSING**

*   Tokenizing
*   Spl chars and digits Removal 
*   Stopword Removal 
*   Case folding 
*   Stemming
*   Creating postings lists











In [2]:
from nltk.stem.snowball import PorterStemmer
doc_map = dict()
vocab = set()
posting_list = defaultdict(list)
stopwords = set(nltk.corpus.stopwords.words("english"))
stemmer = PorterStemmer()

In [3]:
def preprocess(fpath):
  corpus = fpath
  docId=1
  for fname in (os.listdir(corpus)):
    if(os.path.isdir(corpus+"/"+fname)):
      continue
    with open(corpus+"/"+fname,"r") as f:
      txt = f.read()
    #Removing digits and other special chars
    txt = re.sub(re.compile(r"[^a-zA-Z0-9\s']"),"",txt)
    txt = re.sub(re.compile(r"\d"),"",txt)
    # Tokenizing text
    wordlist = nltk.tokenize.word_tokenize(txt)
    
    # Case folding - converting all words to lower case and removing stopwords
    wordlist = [w.lower() for w in wordlist if w not in stopwords]
    
    # Stemming using PorterStemmer algorithm
    wordlist = [stemmer.stem(w) for w in wordlist]
    
    # term list
    dict_terms = list(set(wordlist))
    
    # creating the inverted index data structure
    for t in dict_terms:
      posting_list[t].append(docId)

    # mapping docId==>docPath
    doc_map[docId] = os.path.basename(fname) 

    docId+=1



In [4]:
# unzipping the corpus zip file
!unzip "/content/shakespeares-works_TXT_FolgerShakespeare (1).zip" -d "/content/corpus"

Archive:  /content/shakespeares-works_TXT_FolgerShakespeare (1).zip
  inflating: /content/corpus/much-ado-about-nothing_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/richard-iii_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/the-winters-tale_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/richard-ii_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/henry-vi-part-3_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/the-two-noble-kinsmen_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/venus-and-adonis_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/timon-of-athens_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/the-merchant-of-venice_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/loves-labors-lost_TXT_FolgerShakespeare.txt  
  inflating: /content/corpus/troilus-and-cressida_TXT_FolgerShakespeare.txt  
   creating: /content/corpus/__MACOSX/
  inflating: /content/corpus/__MACOSX/._troilus-and-cressida_TXT_Folge

In [5]:
corpus="/content/corpus" # give path of location of folder containing the docs
preprocess(corpus)
# Creating vocabulary set from posting_list keys
vocab = set(posting_list.keys())

## **Spell check**

In [6]:
# calculates edit distance between 2 words w1 and w2
def edit_dist(w1,w2):
  len1 = len(w1)
  len2 = len(w2)
  if(len1==0):
    return len2
  if(len2==0):
    return len1
  
  dp = [[0 for x in range(len2+1)] for x in range(len1+1)]

  for i in range(len1+1):
    for j in range(len2+1):
      if(i ==0):
        dp[i][j]=j
      elif(j==0):
        dp[i][j]=i
      else:
        if(w1[i-1]!=w2[j-1]):
          dp[i][j] = min({dp[i-1][j]+1,dp[i][j-1]+1,dp[i-1][j-1]+1})
        else:
          dp[i][j] = dp[i-1][j-1]


  return dp[len1][len2]

In [7]:
# Scans the vocab and returns the word having minimum edit distance from w (used to correct the spelling)
def correct_spelling(w):
  if(isOperator(w)):
    return w
  mn_dist = np.inf
  correct_word=""
  for word in vocab:
    dist = edit_dist(w,word)
    if(dist<mn_dist):
      mn_dist=dist
      correct_word=word
  
  return correct_word


## **Wild card Searching**


In [8]:
# Creating permuterm index : maps rotated terms ==> original terms
permuterm = dict()
keys = posting_list.keys()
for k in sorted(keys):
  pkey = k + "$"
  for pos in range(len(pkey),0,-1):
    rot_key = pkey[pos:]+pkey[:pos]
    permuterm[rot_key] = k

In [9]:
def pref_match(w):
  matched_terms=[]
  for keys in permuterm.keys():
    if keys.startswith(w):
      matched_terms.append(permuterm[keys])
  return matched_terms

In [59]:
def match_terms(q):
  word=""
  wa=""
  wb=""
  # splitting a given word with * as delimiter
  parts = q.split('*')
  if len(parts) == 3:
    case=4
  elif parts[1]=='':
    case=1
  elif parts[0]=='':
    case=2
  elif parts[0]!='' and parts[1]!='':
    case=3
  
  if case==4:
    if parts[0] == '':
      case=1
  
  if case==1:
    # terms of form A*
    word = "$"+parts[0]
  elif case==2:
    # terms of form *A
    word = parts[1] + "$"
  elif case==3:
    # terms of form A*B
    word = parts[1] + "$" + parts[0]
  elif case==4:
    # terms of form A*B*C
    wa = parts[2] + "$" + parts[0]
    wb = parts[1]

  possible_terms = ""
  if case!=4:
    lst = pref_match(word)
  else:
    lst = list(set(pref_match(wa)).intersection(set(pref_match(wb)))) 
  
  if(len(lst)==1):
      possible_terms=lst[0]
  elif(len(lst)>0):
    possible_terms += "("
    for i in lst: 
      possible_terms += (" " + i +" | ")
    possible_terms=possible_terms.rstrip(possible_terms[-1])
    possible_terms=possible_terms.rstrip(possible_terms[-1])
    possible_terms += ")"

  #print(word)
  #print(wa)
  #print(wb)
  #print(possible_terms)
  return possible_terms

In [60]:
match_terms("ca*l*ia")

'( cavalleria |  calphurnia |  castalia )'

## **Document matching**

In [62]:
# list of doc ids
docIdlist=doc_map.keys()

In [63]:
# defining precedence of boolean operators
precedence = dict()
precedence['~']=3
precedence['&']=2
precedence['|']=1

In [64]:
# preprocesses the query before finding the matching documents
def query_preprocess(text):
  #text = re.sub(re.compile(r"[^a-zA-Z0-9\s]"),"",text)
  text = re.sub(re.compile(r"\d"),"",text)
  text = text.lower()
  # Tokenizing text
  query_tokens = nltk.tokenize.word_tokenize(text)
  query_tokens = [correct_spelling(w) for w in query_tokens]
  return query_tokens


In [65]:
def isOperator(ch):
  if(ch=='&' or ch=='|' or ch=='~' or ch=='(' or ch==')'):
    return True
  return False

In [66]:
def eval_op(left,right,op):
  result=[]
  if(op=='~'):
    for i in docIdlist:
      if(i not in left):
        result.append(i)
  elif(op=='&'):
    temp = set(right)
    result = [val for val in left if val in temp]
  elif(op=='|'):
    result = list(set(left)|set(right))
  
  return result


In [67]:
# evaluates the query and returns ids of matched docs
def evaluate_and_match(query_tokens):
  operand_stack = deque()
  operator_stack = deque()
  result=[]
  for t in query_tokens:
    if(isOperator(t)):
      if(t=='('):
        operator_stack.append(t)
      elif(t==')'):
        while((operator_stack) and (operator_stack[-1]!='(')):
          op = operator_stack[-1]
          operator_stack.pop()
          if(op=='~'):
            right = operand_stack[-1]
            operand_stack.pop()
            operand_stack.append(eval_op(right,right,op))
          else:
            right = operand_stack[-1]
            operand_stack.pop()
            left = operand_stack[-1]
            operand_stack.pop()
            result = eval_op(left,right,op)
            operand_stack.append(result)
        operator_stack.pop()
      else:
        while((operator_stack) and (operator_stack[-1]=='&' or operator_stack[-1]=='|' or operator_stack[-1]=='~') and (precedence[operator_stack[-1]]>precedence[t])):
          op = operator_stack[-1]
          operator_stack.pop()
          if(op=='~'):
            right = operand_stack[-1]
            operand_stack.pop()
            operand_stack.append(eval_op(right,right,op))
          else:
            right = operand_stack[-1]
            operand_stack.pop()
            left = operand_stack[-1]
            operand_stack.pop()
            result = eval_op(left,right,op)
            operand_stack.append(result)
        operator_stack.append(t)
    else:
      token = stemmer.stem(t)
      operand_stack.append(posting_list[token])
  while(operator_stack):
    op = operator_stack[-1]
    operator_stack.pop()
    if(op=='~'):
      right = operand_stack[-1]
      operand_stack.pop()
      operand_stack.append(eval_op(right,right,op))
    else:
      right = operand_stack[-1]
      operand_stack.pop()
      left = operand_stack[-1]
      operand_stack.pop()
      result = eval_op(left,right,op)
      operand_stack.append(result)
  return operand_stack[0]
    

In [68]:
# Given the matched docId list, returns document names 
def get_docs(lst):
  docs= []
  for i in lst:
    docs.append(doc_map[i])
  
  return docs

## **Querying**
Expects a *well formed query* to be given as input

**Well-formed query:** 

*   Every word/symbol must be space seperated
*   The following symbols represent boolean operators : **AND = & , OR = | , NOT = ~**
*   If parenthesis are used, it must be ensured that they are properly balanced
*   In absence of parenthesis, the precedence order followed by operators is ~ > & > |
* Supports wildcard entries/terms of following formats: A\* , \*A , A\*B , A\*B\*C 




In [119]:
query = input("Enter query: ")
for term in query.split(" "):
  if('*' in term):
    query=query.replace(term,match_terms(term))
q_tokens = query_preprocess(query)

Enter query: brut* & br*oth


In [120]:
q_tokens

['(',
 'brute',
 '|',
 'brutethen',
 '|',
 'brutish',
 '|',
 'brutishgo',
 '|',
 'brutu',
 '|',
 'brutusnoth',
 ')',
 '&',
 '(',
 'broth',
 '|',
 'brutusnoth',
 ')']

In [121]:
ids=evaluate_and_match(q_tokens)

In [122]:
# Returns list of names of documents satisfying the query
get_docs(ids)

['julius-caesar_TXT_FolgerShakespeare.txt',
 'the-merchant-of-venice_TXT_FolgerShakespeare.txt',
 'henry-v_TXT_FolgerShakespeare.txt']